In [1]:
!pip install stanza
import stanza
import sqlite3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 45.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12


In [12]:
con = sqlite3.connect("proyectoPura.db")
cur = con.cursor()

query = """
SELECT Título_Obra FROM OBRAS
UNION ALL
SELECT Título_Complemento FROM COMPLEMENTOS
UNION ALL
SELECT Título_Sección FROM SECCIONES_PUBLICADAS
"""

cur.execute(query)
titles = cur.fetchall()
#print(titles)

titles_list = []
for title in titles:
  if title[0] is not None:
    titles_list.append(title[0])
  if title[0] is None:
    titles_list.append("NULL") #we take into account "NULL" titles, to keep the same structure as original DB.
print(titles_list)

con.close()

['Al Sr. D. R. P. Enviándole mis obras', 'La mujer piadosa (Fragmentos)', 'Desvarío', 'Soneto. "¡Gloria y pobreza! El inmortal Quintana..."', 'Balada. La despedida de una hija', 'Un viaje a la Luna. Prólogo: La pesadilla de Mocquet', 'En el álbum de la señora M…', 'La primavera', 'La perejilera', 'Un viaje a la Luna. (Continuación)', 'Al señor Don B. M.', 'A orillas del Manzanares. La súplica', 'Un viaje a la Luna. (Continuación)', 'Fragmentos de la Rosa de Alejandra', 'Gente morena', 'Un viaje a la Luna. (Continuación)', 'Al excmo. Sr. D. Agustín Esteban Collantes, en el día de su santo', 'A la niña Margarita Aurora de Wilson y Serrano, el día de su cumpleaños', 'Costumbres del siglo XIX. I', 'Margarita. Novela bretona', 'Los escondidos y la tapada. Fragmentos', 'A mi apreciable amiga la Señora doña M. Del Pilar Sinués de Marco en su álbum', 'Costumbres del siglo XIX.II', 'Margarita. Novela bretona. (Continuación)', 'A la niña Margarita Aurora de Wilson y Serrano', 'A María', 'A su Al

In [5]:
#we start complete formatting before NER:

import re

def subtitle_format(matchobj):
  match = matchobj.group(0)
  if re.fullmatch(r'\s*"\s*-\s*', match):
    return '" | '
  elif re.fullmatch(r'\s*-\s*"\s*', match):
    return ' | "'
  elif re.fullmatch(r'\s*-\s*"\s*-\s*', match):
    return '" | '
  elif re.fullmatch(r'\s*\[\.\.\.\]\s*"\s*-\s*', match):
    submatch = re.fullmatch(r'(\s*)\[\.\.\.\]\s*"\s*-\s*', match)
    return submatch.group(1)+'[...]" | '

def formatting_subtitles(text):
    subtit_re = r'\s*"\s*-\s*|\s*-\s*"\s*|\s*-\s*"\s*-\s*|\s*\[\.\.\.\]\s*"\s*-\s*'
    new_text = re.sub(subtit_re, subtitle_format, text)
    return new_text

def formatting_century(text):
  re_century_1 = r'(siglo|siglos)\s*(I|II|III|IV|V|VI|VII|VIII|IX|X|XI|XII|XIII|XIV|XV|XVI|XVII|XVIII|XIX|XX|XXI)\.\s*'
  re_century_2 = r'(siglo|siglos)\s*(I|II|III|IV|V|VI|VII|VIII|IX|X|XI|XII|XIII|XIV|XV|XVI|XVII|XVIII|XIX|XX|XXI)\s+\.'
  if re.search(re_century_1, text):
    return re.sub(re_century_1, r'[("\1 \2", "DATE")] ', text).strip()
  if re.search(re_century_2, text):
    return re.sub(re_century_2, r'[("\1 \2", "DATE")]', text).strip()
  else:
    return text

encontrar_subtit = [formatting_subtitles(row) for row in titles_list]
list_titsub = [row for row in encontrar_subtit if " | " in row]

#only this exceptions do not adjust to pattern (2 rows):
excepcion_1 = """Piquer-Oda "Alzad vuestras"""
excepcion_2 = """Matrimonios por anuncios-Los baños de Biarritz"""
def subadhoc(matchobj):
  return ' "'+ matchobj.group(0)[1:]
for i, row in enumerate(encontrar_subtit):
  if excepcion_1 in row:
    encontrar_subtit[i] = row.replace('Piquer-Oda "Alzad', 'Piquer-Oda. "Alzad') #Mismo formato que algunos sonetos.
  if excepcion_2 in row:
    encontrar_subtit[i]= row.replace('-', ' | ')
    #encontrar_subtit[i] = re.sub(r'[^|]+\]"', subadhoc, encontrar_subtit[i])##N: añadido mío (con la definición de subadhoc)
#print(encontrar_subtit[249])
#print(encontrar_subtit[207])

encontrar_century = [formatting_century(row).strip() for row in encontrar_subtit]

encontrar_century[:30]

['Al Sr. D. R. P. Enviándole mis obras',
 'La mujer piadosa (Fragmentos)',
 'Desvarío',
 'Soneto. "¡Gloria y pobreza! El inmortal Quintana..."',
 'Balada. La despedida de una hija',
 'Un viaje a la Luna. Prólogo: La pesadilla de Mocquet',
 'En el álbum de la señora M…',
 'La primavera',
 'La perejilera',
 'Un viaje a la Luna. (Continuación)',
 'Al señor Don B. M.',
 'A orillas del Manzanares. La súplica',
 'Un viaje a la Luna. (Continuación)',
 'Fragmentos de la Rosa de Alejandra',
 'Gente morena',
 'Un viaje a la Luna. (Continuación)',
 'Al excmo. Sr. D. Agustín Esteban Collantes, en el día de su santo',
 'A la niña Margarita Aurora de Wilson y Serrano, el día de su cumpleaños',
 'Costumbres del [("siglo XIX", "DATE")] I',
 'Margarita. Novela bretona',
 'Los escondidos y la tapada. Fragmentos',
 'A mi apreciable amiga la Señora doña M. Del Pilar Sinués de Marco en su álbum',
 'Costumbres del [("siglo XIX", "DATE")] II',
 'Margarita. Novela bretona. (Continuación)',
 'A la niña Margari

In [6]:
#we continue preparing each title to process it through model (ner)

#def clean_per_title(title):
  #cleaned_text = re.sub(
    #r'\s*\(\s*(Continuación|Fragmentos?|Conclusión|Conclusiones?|Conclusion)\.?\s*\)\s*', #not now, mayber after processing
    #' ',
    #title)
  #cleaned_text = re.sub(r'\s*"\s*|\s*\[\s*\.\s*\.\s*\.\s*\]\s*|\s*\[\s*\u2026\s*\]\s*', ' ', cleaned_text)
  #return cleaned_text.strip()

#cleaned_rows = [clean_per_title(row) for row in encontrar_subtit]

#print(cleaned_rows)
titles_to_NER = [row.split(" | ") if " | " in row else [row] for row in encontrar_century]

for row in titles_to_NER[:30]:
  print(row, "\n")

['Al Sr. D. R. P. Enviándole mis obras'] 

['La mujer piadosa (Fragmentos)'] 

['Desvarío'] 

['Soneto. "¡Gloria y pobreza! El inmortal Quintana..."'] 

['Balada. La despedida de una hija'] 

['Un viaje a la Luna. Prólogo: La pesadilla de Mocquet'] 

['En el álbum de la señora M…'] 

['La primavera'] 

['La perejilera'] 

['Un viaje a la Luna. (Continuación)'] 

['Al señor Don B. M.'] 

['A orillas del Manzanares. La súplica'] 

['Un viaje a la Luna. (Continuación)'] 

['Fragmentos de la Rosa de Alejandra'] 

['Gente morena'] 

['Un viaje a la Luna. (Continuación)'] 

['Al excmo. Sr. D. Agustín Esteban Collantes, en el día de su santo'] 

['A la niña Margarita Aurora de Wilson y Serrano, el día de su cumpleaños'] 

['Costumbres del [("siglo XIX", "DATE")] I'] 

['Margarita. Novela bretona'] 

['Los escondidos y la tapada. Fragmentos'] 

['A mi apreciable amiga la Señora doña M. Del Pilar Sinués de Marco en su álbum'] 

['Costumbres del [("siglo XIX", "DATE")] II'] 

['Margarita. Novela

In [7]:
#first processing, we extract NER
def processing_1(text, nlp_model):
  doc = nlp_model(text)
  if text == "NULL":
    return [("NULL")]
  if "[(" not in text:
    return [(ent.text, ent.type) for ent in doc.entities]
  if "[(" in text:
    special_categories = re.findall(r'\[\(.*\)\]', text)
    parsing_text = re.split(r'\[\(.*\)\]', text)
    exception = []
    for part in parsing_text:
      if part.startswith("[(") and part.endswith("])"):
        exception.append(part)
      else:
        doc = nlp_model(part)
        exception.extend([(ent.text, ent.type) for ent in doc.entities])
    return exception

nlp_es = stanza.Pipeline(lang="es", processors="tokenize,mwt,pos,lemma,ner,depparse")

processed_ent_es = [[processing_1(el, nlp_es) for el in lista] for lista in titles_to_NER]

for element in processed_ent_es:
  print(element, "\n")



INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json


INFO:stanza:Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |
| ner       | conll02           |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


[[('Sr. D. R. P.', 'MISC')]] 

[[('Fragmentos', 'MISC')]] 

[[('Desvarío', 'MISC')]] 

[[('Soneto', 'PER'), ('inmortal Quintana...', 'MISC')]] 

[[('Balada', 'PER')]] 

[[('Luna', 'MISC'), ('Mocquet', 'PER')]] 

[[('señora M…', 'PER')]] 

[[]] 

[[]] 

[[('Luna', 'MISC')]] 

[[('Don B. M.', 'PER')]] 

[[('Manzanares', 'LOC')]] 

[[('Luna', 'MISC')]] 

[[('de la Rosa de Alejandra', 'PER')]] 

[[]] 

[[('Luna', 'MISC')]] 

[[('Sr. D. Agustín Esteban Collantes', 'MISC')]] 

[[('Margarita Aurora de Wilson', 'PER'), ('Serrano', 'PER')]] 

[[('I', 'PER')]] 

[[('Margarita', 'PER')]] 

[[]] 

[[('Señora doña M. Del Pilar Sinués de Marco', 'PER')]] 

[[('II', 'MISC')]] 

[[('Margarita', 'PER')]] 

[[('Margarita Aurora de Wilson', 'PER'), ('Serrano', 'PER')]] 

[[('María', 'PER')]] 

[[('Alteza Real', 'PER'), ('Príncipe de Asturias', 'PER')]] 

[[('Margarita', 'PER')]] 

[[]] 

[[('Soneto', 'PER')]] 

[[('SS. MM. II', 'PER')]] 

[[('El matrimonio de la Princesa real de Inglaterra [...]', 'MISC'

In [8]:
from ast import Pass
import copy

titles_to_NER_COPY = copy.deepcopy(titles_to_NER)

for index, row in enumerate(processed_ent_es):
  if index in [18, 22]:  # Problematic samples, we create exception
    continue
  else:
    for sub_index, entidad in enumerate(row):
      for tuple_item in entidad:
        if tuple_item is None:
          pass
        if isinstance(tuple_item, str):
          pass
        if isinstance(tuple_item, tuple):
          entity_str = tuple_item[0]
          replacement_str = f"[('{tuple_item[0]}', '{tuple_item[1]}')]"
          titles_to_NER_COPY[index][sub_index] = re.sub(re.escape(entity_str), replacement_str, titles_to_NER_COPY[index][sub_index])
        #except TypeError:
          #pass
        #except IndexError:
          #pass

for el in titles_to_NER_COPY[:50]:
    print(el, "\n")

["Al [('Sr. D. R. P.', 'MISC')] Enviándole mis obras"] 

["La mujer piadosa ([('Fragmentos', 'MISC')])"] 

["[('Desvarío', 'MISC')]"] 

['[(\'Soneto\', \'PER\')]. "¡Gloria y pobreza! El [(\'inmortal Quintana...\', \'MISC\')]"'] 

["[('Balada', 'PER')]. La despedida de una hija"] 

["Un viaje a la [('Luna', 'MISC')]. Prólogo: La pesadilla de [('Mocquet', 'PER')]"] 

["En el álbum de la [('señora M…', 'PER')]"] 

['La primavera'] 

['La perejilera'] 

["Un viaje a la [('Luna', 'MISC')]. (Continuación)"] 

["Al señor [('Don B. M.', 'PER')]"] 

["A orillas del [('Manzanares', 'LOC')]. La súplica"] 

["Un viaje a la [('Luna', 'MISC')]. (Continuación)"] 

["Fragmentos [('de la Rosa de Alejandra', 'PER')]"] 

['Gente morena'] 

["Un viaje a la [('Luna', 'MISC')]. (Continuación)"] 

["Al excmo. [('Sr. D. Agustín Esteban Collantes', 'MISC')], en el día de su santo"] 

["A la niña [('Margarita Aurora de Wilson', 'PER')] y [('Serrano', 'PER')], el día de su cumpleaños"] 

['Costumbres del [("sigl

In [9]:
#we've gotten categories related to NER processing, after including entities [(x,y)] in each row/title
#we proceed with the second part of processing (tok,lemma,pos,depp) keeping NER categories without processing.

def processing_2(text, nlp_model):
  doc = nlp_model(text)
  if text == "NULL":
    return [("NULL")]
  if "[(" not in text:
    return [(word.text, word.lemma, word.upos) for sent in doc.sentences for word in sent.words]
  if "[(" in text:
    special_categories = re.findall(r'\[\(.*\)\]', text)
    parsing_text = re.split(r'\[\(.*\)\]', text)
    exception = []
    for part in parsing_text:
      if part.startswith("[(") and part.endswith(")]"):
        exception.append(part)

      else:
        doc = nlp_model(part)
        exception.extend([(word.text, word.lemma, word.upos) for sent in doc.sentences for word in sent.words])
    return exception

nlp_es = stanza.Pipeline(lang="es", processors="tokenize,mwt,pos,lemma")

pro_lem = [[processing_2(el, nlp_es) for el in lista] for lista in titles_to_NER_COPY]

for element in pro_lem[:50]:
  print(element, "\n")

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: es (Spanish):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!


[[('A', 'a', 'ADP'), ('el', 'el', 'DET'), ('Enviando', 'enviar', 'VERB'), ('le', 'él', 'PRON'), ('mis', 'mi', 'DET'), ('obras', 'obra', 'NOUN')]] 

[[('La', 'el', 'DET'), ('mujer', 'mujer', 'NOUN'), ('piadosa', 'piadoso', 'ADJ'), ('(', '(', 'PUNCT'), (')', ')', 'PUNCT')]] 

[[]] 

[[('"', '"', 'PUNCT')]] 

[[('.', '.', 'PUNCT'), ('La', 'el', 'DET'), ('despedida', 'despedida', 'NOUN'), ('de', 'de', 'ADP'), ('una', 'uno', 'DET'), ('hija', 'hija', 'NOUN')]] 

[[('Un', 'uno', 'DET'), ('viaje', 'viaje', 'NOUN'), ('a', 'a', 'ADP'), ('la', 'el', 'DET')]] 

[[('En', 'en', 'ADP'), ('el', 'el', 'DET'), ('álbum', 'álbum', 'NOUN'), ('de', 'de', 'ADP'), ('la', 'el', 'DET')]] 

[[('La', 'el', 'DET'), ('primavera', 'primavera', 'NOUN')]] 

[[('La', 'el', 'DET'), ('perejilera', 'perejilera', 'NOUN')]] 

[[('Un', 'uno', 'DET'), ('viaje', 'viaje', 'NOUN'), ('a', 'a', 'ADP'), ('la', 'el', 'DET'), ('.', '.', 'PUNCT'), ('(', '(', 'PUNCT'), ('Continuación', 'continuación', 'NOUN'), (')', ')', 'PUNCT')]] 

[

In [10]:
pro_lem_cleaned = copy.deepcopy(pro_lem)

for i_element, element in enumerate(pro_lem):
  for i_titulo, titulo in enumerate(element):
    pro_lem_cleaned[i_element][i_titulo] = [tupla for tupla in titulo if isinstance(tupla, tuple) and tupla[2] in ('NOUN', 'PROPN', 'ADJ', 'VERB', 'ADV', 'INTJ')]
      #pro_lem_cleaned[i_element][i_titulo] = [tupla for tupla in titulo if isinstance(tupla, tuple) and tupla[2] in ('NOUN', 'PROPN', 'ADJ', 'VERB', 'ADV', 'INTJ') or isinstance(tupla, str)]

In [11]:
for element in pro_lem_cleaned[:100]:
  print(element, "\n")

[[('Enviando', 'enviar', 'VERB'), ('obras', 'obra', 'NOUN')]] 

[[('mujer', 'mujer', 'NOUN'), ('piadosa', 'piadoso', 'ADJ')]] 

[[]] 

[[]] 

[[('despedida', 'despedida', 'NOUN'), ('hija', 'hija', 'NOUN')]] 

[[('viaje', 'viaje', 'NOUN')]] 

[[('álbum', 'álbum', 'NOUN')]] 

[[('primavera', 'primavera', 'NOUN')]] 

[[('perejilera', 'perejilera', 'NOUN')]] 

[[('viaje', 'viaje', 'NOUN'), ('Continuación', 'continuación', 'NOUN')]] 

[[('señor', 'señor', 'NOUN')]] 

[[('orillas', 'orilla', 'NOUN'), ('súplica', 'súplica', 'NOUN')]] 

[[('viaje', 'viaje', 'NOUN'), ('Continuación', 'continuación', 'NOUN')]] 

[[('Fragmentos', 'fragmentos', 'NOUN')]] 

[[('Gente', 'gente', 'NOUN'), ('morena', 'moreno', 'ADJ')]] 

[[('viaje', 'viaje', 'NOUN'), ('Continuación', 'continuación', 'NOUN')]] 

[[('excmo', 'excmo', 'NOUN'), ('día', 'día', 'NOUN'), ('santo', 'santo', 'NOUN')]] 

[[('niña', 'niña', 'NOUN'), ('día', 'día', 'NOUN'), ('cumpleaños', 'cumpleaños', 'NOUN')]] 

[[('Costumbres', 'costumbres', '